In [1]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from rwr_functions import *
from constants import *

from sknetwork.data import from_edge_list
from networkx import pagerank, from_pandas_edgelist


# Create supra-adjacency matrix

In [7]:
def load_multilayer_graph(selected_networks):
    gene_nets = {}

    # Intranetwork edges
    intra_edges = pd.DataFrame()
    for net in selected_networks:
        print("Loading {} graph".format(net))
        df = pd.read_csv(network_files[net], dtype={'node1': str, 'node2': str})[["node1", "node2"]].astype(str)    
        graph_nodes = set(df["node1"]).union(set(df["node2"]))
        for nd in graph_nodes:
            if nd in gene_nets.keys():
                gene_nets[nd].append(net)
            else:
                gene_nets[nd] = [net]

        df["node1"] = net+"_"+df["node1"]
        df["node2"] = net+"_"+df["node2"]
        intra_edges = pd.concat((intra_edges, df))

    # Find inter-network edges
    inter_edges = []
    tot_multigraph_genes = 0
    for gn, nn in gene_nets.items():
        if len(nn)<2:
            continue
        tot_multigraph_genes += 1
        for i in range(0, len(nn)-1):
            for j in range(i+1, len(nn)):
    #             print(nn[i], nn[j])
                inter_edges.append((nn[i]+"_"+gn, nn[j]+"_"+gn))
    inter_edges = pd.DataFrame(inter_edges, columns=["node1", "node2"])

    # Complete Supra-matrix
    supadj_edges = pd.concat((intra_edges, inter_edges))
    
    
    # graph = from_edge_list(supadj_edges.values.astype(str))
    graph = from_pandas_edgelist(supadj_edges, source="node1", target="node2")
    return graph


def init_multilayer_scores(graph, data):
    # node2idx = {str(n): i for i, n in enumerate(graph.names)}
    # idx2node = {v: k for k, v in node2idx.items()}

    ncbi2gene = dict(zip(data.NCBI_id, data.Gene))
    ncbi_genes = set(data.NCBI_id)
    pegasus_scores = dict(zip(data.NCBI_id, data.Score))
    pagerank_seeds = {}
    for node in graph.nodes: #graph["names"].astype(str):
        node_gene = node.split("_", 1)[-1]
        if node_gene in ncbi_genes:
            # pagerank_seeds[node2idx[node]] = pegasus_scores[node_gene]
            pagerank_seeds[node] = pegasus_scores[node_gene]
        else:
            # pagerank_seeds[node2idx[node]] = 0.0
            pagerank_seeds[node] = 0.0
    return pagerank_seeds


In [8]:
graph = load_multilayer_graph(NETWORKS)

Loading BioPlex3 graph
Loading HumanNet graph
Loading PCNet graph
Loading ProteomeHD graph
Loading STRING graph


In [10]:
results = []


for disease in tqdm(DISEASES):
    pegasus_data = load_pegasus_results(disease)
    pagerank_seeds = init_multilayer_scores(graph, pegasus_data)
    gene_seeds_ncbi, ncbi_targets = load_seeds_and_targets(disease)
    ncbi_targets = list(set(ncbi_targets))
    for alpha in tqdm(ALPHAS):
        rwr_scores = perform_rwr_nx(alpha, graph, pagerank_seeds)
        rwr_results = process_rwr_results_nx(rwr_scores, graph, pegasus_data, pagerank_seeds) 
        rwr_results[["Network", "Gene NCBI ID"]] = rwr_results["Gene NCBI ID"].str.split("_", n=1, expand=True)
        rwr_results = rwr_results.drop_duplicates("Gene NCBI ID", keep="first")
        for K in Ks:
            metrics = calculate_metrics(rwr_results, K, gene_seeds_ncbi, ncbi_targets, "Multilayer", alpha, disease, scoring="Score")
            results.append(metrics)
                
results = pd.concat(results)
results

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

/tmp/ipykernel_114787/2745808437.py:12: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  rwr_results[["Network", "Gene NCBI ID"]] = rwr_results["Gene NCBI ID"].str.split("_", 1, expand=True)
/tmp/ipykernel_114787/2745808437.py:12: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  rwr_results[["Network", "Gene NCBI ID"]] = rwr_results["Gene NCBI ID"].str.split("_", 1, expand=True)
/tmp/ipykernel_114787/2745808437.py:12: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  rwr_results[["Network", "Gene NCBI ID"]] = rwr_results["Gene NCBI ID"].str.split("_", 1, expand=True)
/tmp/ipykernel_114787/2745808437.py:12: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keywor

  0%|          | 0/100 [00:00<?, ?it/s]

/tmp/ipykernel_114787/2745808437.py:12: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  rwr_results[["Network", "Gene NCBI ID"]] = rwr_results["Gene NCBI ID"].str.split("_", 1, expand=True)
/tmp/ipykernel_114787/2745808437.py:12: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  rwr_results[["Network", "Gene NCBI ID"]] = rwr_results["Gene NCBI ID"].str.split("_", 1, expand=True)
/tmp/ipykernel_114787/2745808437.py:12: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  rwr_results[["Network", "Gene NCBI ID"]] = rwr_results["Gene NCBI ID"].str.split("_", 1, expand=True)
/tmp/ipykernel_114787/2745808437.py:12: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keywor

  0%|          | 0/100 [00:00<?, ?it/s]

/tmp/ipykernel_114787/2745808437.py:12: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  rwr_results[["Network", "Gene NCBI ID"]] = rwr_results["Gene NCBI ID"].str.split("_", 1, expand=True)
/tmp/ipykernel_114787/2745808437.py:12: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  rwr_results[["Network", "Gene NCBI ID"]] = rwr_results["Gene NCBI ID"].str.split("_", 1, expand=True)
/tmp/ipykernel_114787/2745808437.py:12: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  rwr_results[["Network", "Gene NCBI ID"]] = rwr_results["Gene NCBI ID"].str.split("_", 1, expand=True)
/tmp/ipykernel_114787/2745808437.py:12: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keywor

,Network,Alpha,Metric,K,Value,Method,Disease
0,Multilayer,0.000,Average Precision,20,0.664748,Score,asthma
0,Multilayer,0.000,Average Precision,50,0.535414,Score,asthma
0,Multilayer,0.000,Average Precision,100,0.466092,Score,asthma
0,Multilayer,0.001,Average Precision,20,0.664748,Score,asthma
0,Multilayer,0.001,Average Precision,50,0.535414,Score,asthma
...,...,...,...,...,...,...,...
0,Multilayer,0.890,Average Precision,50,0.045813,Score,schizophrenia
0,Multilayer,0.890,Average Precision,100,0.050576,Score,schizophrenia
0,Multilayer,0.900,Average Precision,20,0.000000,Score,schizophrenia
0,Multilayer,0.900,Average Precision,50,0.046653,Score,schizophrenia


In [11]:
results.to_csv("outputs/multilayer_metrics.csv", index=False)

In [12]:
results

,Network,Alpha,Metric,K,Value,Method,Disease
0,Multilayer,0.000,Average Precision,20,0.664748,Score,asthma
0,Multilayer,0.000,Average Precision,50,0.535414,Score,asthma
0,Multilayer,0.000,Average Precision,100,0.466092,Score,asthma
0,Multilayer,0.001,Average Precision,20,0.664748,Score,asthma
0,Multilayer,0.001,Average Precision,50,0.535414,Score,asthma
...,...,...,...,...,...,...,...
0,Multilayer,0.890,Average Precision,50,0.045813,Score,schizophrenia
0,Multilayer,0.890,Average Precision,100,0.050576,Score,schizophrenia
0,Multilayer,0.900,Average Precision,20,0.000000,Score,schizophrenia
0,Multilayer,0.900,Average Precision,50,0.046653,Score,schizophrenia
